In [1]:
print("hello world")

hello world


In [2]:
import gymnasium as gym
env = gym.make('Taxi-v3', render_mode='human')
env.reset()

(389, {'prob': 1.0, 'action_mask': array([1, 1, 0, 1, 0, 0], dtype=int8)})

In [3]:
import pygame
observation, info = env.reset()
for _ in range(1000):
   pygame.event.get()
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()

env.close()

KeyboardInterrupt: 

In [4]:
env.close()

In [ ]:
import numpy as np
import pygame

import gymnasium as gym
from gymnasium import spaces


class GridWorldEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None, size=5):
        self.size = size  # The size of the square grid
        self.window_size = 512  # The size of the PyGame window

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent1": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "agent2": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2,), dtype=int),
            }
        )

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.MultiDiscrete([4,4])

        """
        The following dictionary maps abstract actions from `self.action_space` to
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

    def _get_obs(self):
        return {"agent1": self._agent_location,  "target": self._target_location}
    
    def _get_info(self):
        return {
            "distance": np.linalg.norm(
                self._agent_location - self._target_location, ord=1
            )
        }
    
    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        self._agent_location = self.np_random.integers(0, self.size, size=2, dtype=int)

        # We will sample the target's location randomly until it does not coincide with the agent's location
        self._target_location = self._agent_location
        while np.array_equal(self._target_location, self._agent_location):
            self._target_location = self.np_random.integers(
                0, self.size, size=2, dtype=int
            )

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def step(self, action):
        # Map the action (element of {0,1,2,3}) to the direction we walk in
        direction = self._action_to_direction[action]
        # We use `np.clip` to make sure we don't leave the grid
        self._agent_location = np.clip(
            self._agent_location + direction, 0, self.size - 1
        )
        # An episode is done iff the agent has reached the target
        terminated = np.array_equal(self._agent_location, self._target_location)
        reward = 1 if terminated else 0  # Binary sparse rewards
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info
    
    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (self.window_size, self.window_size)
            )
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.size
        )  # The size of a single grid square in pixels

        # First we draw the target
        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * self._target_location,
                (pix_square_size, pix_square_size),
            ),
        )
        # Now we draw the agent
        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (self._agent_location + 0.5) * pix_square_size,
            pix_square_size / 3,
        )

        # Finally, add some gridlines
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        if self.render_mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()
    
    

In [23]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pygame

class TwoAgentGridworldEnv(gym.Env):
    metadata = {'render_modes': ['human', 'print'], "render_fps": 4}
    
    def __init__(self, render_mode=None, grid_size=(10, 10)):
        super(TwoAgentGridworldEnv, self).__init__()
        self.size = grid_size[0]  # The size of the square grid
        self.window_size = 800  # The size of the PyGame window
        
        self.grid_size = grid_size
        self.action_space = spaces.MultiDiscrete([4, 4])  # 4 possible actions for each of the two agents
        self.observation_space = spaces.Tuple((
            spaces.Box(low=0, high=grid_size[0]-1, shape=(2,), dtype=np.int32),
            spaces.Box(low=0, high=grid_size[1]-1, shape=(2,), dtype=np.int32)
        ))
        
        
        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode
        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None
        # Reset the environment and start
        self.reset()

    def reset(self):
        self.agent_positions = [
            np.array([0, 0]),  # Agent 1 starts at top-left corner
            np.array([self.grid_size[0]-1, self.grid_size[1]-1])  # Agent 2 starts at bottom-right corner
        ]
        return self._get_obs()

    def _get_obs(self):
        return {'agent1': self.agent_positions[0], 'agent2': self.agent_positions[1]}

    def step(self, action):
        # Define the movements corresponding to each action
        movements = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right
        rewards = 0
        
        # Update the positions of both agents
        for i, act in enumerate(action):
            movement = movements[act]
            new_position = self.agent_positions[i] + movement
            # Ensure the new position is within bounds
            new_position = np.clip(new_position, [0, 0], [self.grid_size[0]-1, self.grid_size[1]-1])
            self.agent_positions[i] = new_position
        
        # Placeholder for terminal state and rewards
        done = False
        reward = 0
        
        # If the agents meet at the same position, we can assign a reward or consider it a terminal state
        if np.array_equal(self.agent_positions[0], self.agent_positions[1]):
            done = True
            reward = 10  # Example reward for meeting at the same position
        
        obs = self._get_obs()
        return obs, reward, done, False, {}

    def render(self):
        if self.render_mode == 'print':
            grid = np.zeros(self.grid_size)
            grid[tuple(self.agent_positions[0])] = 1  # Mark the position of the first agent
            grid[tuple(self.agent_positions[1])] = 2  # Mark the position of the second agent
            print(grid)
        elif self.render_mode == 'human':
            if self.window is None: # Initialize pygame if it is not initialized
                pygame.init()
                pygame.display.init()
                self.window = pygame.display.set_mode(
                    (self.window_size, self.window_size)
                )
            if self.clock is None:
                self.clock = pygame.time.Clock()
            
            # Fill the canvas
            canvas = pygame.Surface((self.window_size, self.window_size))
            canvas.fill((255, 255, 255))
            pix_square_size = (
                self.window_size / self.size
            )  # The size of a single grid square in pixels


            # First we draw agent1 square
            pygame.draw.rect(
                canvas,
                (255, 0, 0),
                pygame.Rect(
                    pix_square_size * self.agent_positions[0],
                    (pix_square_size, pix_square_size),
                ),
            )
            # Now we draw the agent2 circle
            pygame.draw.circle(
                canvas,
                (0, 0, 255),
                (self.agent_positions[1] + 0.5) * pix_square_size,
                pix_square_size / 3,
            )
            # Finally, add some gridlines
            for x in range(self.size + 1):
                pygame.draw.line(
                    canvas,
                    0,
                    (0, pix_square_size * x),
                    (self.window_size, pix_square_size * x),
                    width=3,
                )
                pygame.draw.line(
                    canvas,
                    0,
                    (pix_square_size * x, 0),
                    (pix_square_size * x, self.window_size),
                    width=3,
                )
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
            
    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

In [24]:
# Register the environment
gym.envs.registration.register(
    id='TwoAgentGridworld-v0',
    entry_point=TwoAgentGridworldEnv,
    max_episode_steps=100,
)

# Example of creating and using the environment
env = gym.make('TwoAgentGridworld-v0', render_mode='human')
env.metadata['render_fps'] = 8
obs = env.reset()
env.render()

done = False
while not done:
    action = env.action_space.sample()
    obs, reward, done, _,  info = env.step(action)
    env.render()
    print(f"Obs: {obs}, Reward: {reward}, Done: {done}")

Obs: {'agent1': array([0, 1]), 'agent2': array([9, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([1, 1]), 'agent2': array([8, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([1, 2]), 'agent2': array([7, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([0, 2]), 'agent2': array([7, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([1, 2]), 'agent2': array([7, 8])}, Reward: 0, Done: False
Obs: {'agent1': array([1, 1]), 'agent2': array([8, 8])}, Reward: 0, Done: False
Obs: {'agent1': array([2, 1]), 'agent2': array([7, 8])}, Reward: 0, Done: False
Obs: {'agent1': array([1, 1]), 'agent2': array([8, 8])}, Reward: 0, Done: False
Obs: {'agent1': array([0, 1]), 'agent2': array([8, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([1, 1]), 'agent2': array([8, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([1, 0]), 'agent2': array([9, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([2, 0]), 'agent2': array([9, 9])}, Reward: 0, Done: False
Obs: {'agent1': array([3, 0]), 'agent2':

In [17]:
env.close()